# MLflow Experiment Tracking Demo

This notebook demonstrates how to use MLflow for experiment tracking in the domain suggestion project.

In [ ]:
import mlflow
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set MLflow tracking URI
mlflow.set_tracking_uri("file:///tmp/mlflow-tracking")

print("MLflow tracking URI:", mlflow.get_tracking_uri())

In [ ]:
# List experiments
from mlflow.entities import ViewType

experiments = mlflow.list_experiments()
print("Available experiments:")
for exp in experiments:
    print(f"- {exp.name} (ID: {exp.experiment_id})")

In [ ]:
# Load evaluation data
eval_file = '../evaluation/enhanced_comprehensive_evaluation.json'
if os.path.exists(eval_file):
    with open(eval_file, 'r') as f:
        eval_data = json.load(f)
    print("Loaded evaluation data successfully!")
    print("Keys in evaluation data:", list(eval_data.keys()))
else:
    print(f"Evaluation data not found at {eval_file}")
    # Load the original evaluation data as fallback
    original_eval_file = '../evaluation/comprehensive_evaluation.json'
    if os.path.exists(original_eval_file):
        with open(original_eval_file, 'r') as f:
            eval_data = json.load(f)
        print("Loaded original evaluation data successfully!")
    else:
        eval_data = {}

In [ ]:
# Display key metrics
if eval_data:
    print("\nKey Metrics:")
    print(f"Average Confidence: {eval_data.get('average_confidence', 0):.3f}")
    print(f"Domain Diversity: {eval_data.get('domain_diversity', 0):.3f}")
    print(f"Average Domain Length: {eval_data.get('average_domain_length', 0):.2f}")
    print(f"Total Suggestions: {eval_data.get('total_suggestions', 0)}")
    print(f"Unique Domains: {eval_data.get('unique_domains', 0)}")

In [ ]:
# Visualize extension distribution
if 'extension_distribution' in eval_data:
    extensions = eval_data['extension_distribution']
    
    plt.figure(figsize=(10, 6))
    extensions_df = pd.DataFrame(list(extensions.items()), columns=['Extension', 'Count'])
    sns.barplot(data=extensions_df, x='Extension', y='Count', palette='viridis')
    plt.title('Domain Extension Distribution')
    plt.xlabel('Extension')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize confidence distribution
if 'confidence_distribution' in eval_data:
    confidence = eval_data['confidence_distribution']
    
    plt.figure(figsize=(10, 6))
    confidence_df = pd.DataFrame(list(confidence.items()), columns=['Confidence Range', 'Count'])
    sns.barplot(data=confidence_df, x='Confidence Range', y='Count', palette='plasma')
    plt.title('Confidence Score Distribution')
    plt.xlabel('Confidence Range')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Start a new MLflow run for this notebook
mlflow.set_experiment("domain_suggestion_notebook_analysis")

with mlflow.start_run(run_name="notebook_analysis_run"):
    # Log parameters
    mlflow.log_params({
        "analysis_type": "notebook_demo",
        "dataset_size": eval_data.get('total_suggestions', 0)
    })
    
    # Log metrics
    if eval_data:
        metrics = {
            "avg_confidence": eval_data.get('average_confidence', 0),
            "domain_diversity": eval_data.get('domain_diversity', 0),
            "avg_domain_length": eval_data.get('average_domain_length', 0)
        }
        mlflow.log_metrics(metrics)
    
    # Log this notebook as an artifact
    mlflow.log_artifact("../notebooks/03_mlflow_demo.ipynb")
    
    run_id = mlflow.active_run().info.run_id
    print(f"Logged notebook analysis to MLflow run: {run_id}")

print("\nTo view MLflow UI, run in terminal:")
print("mlflow ui --backend-store-uri file:///tmp/mlflow-tracking")